#### Using intersection BOW and difference BOW as features for Naive Bayes, SGD.

#### Probabilities generated are then used as features for XGBoost later

In [4]:
from __future__ import unicode_literals
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
item_info = pd.read_csv('../ItemInfo_train.csv', encoding='utf-8', usecols=['itemID'])

In [6]:
desc_clean = pd.read_csv('./ItemInfo_train5.csv', encoding='utf-8',usecols=['description_x_clean','description_y_clean'], converters={'description_x_clean':lambda x: x.strip(u'[]').split(', '),
                                                                                                                                      'description_y_clean':lambda x: x.strip(u'[]').split(', ')})

In [7]:
item_pairs = pd.read_csv('../ItemPairs_train.csv', encoding='utf-8', usecols= ['itemID_1', 'itemID_2'])
item_pairs.rename(columns={'itemID_1':'itemID'}, inplace=True)

In [8]:
df = pd.merge(item_info, item_pairs, on='itemID')
item_info.rename(columns={'itemID': 'itemID_2'}, inplace=True)
df = pd.merge(df, item_info, on='itemID_2')

In [9]:
df = pd.concat([df,desc_clean], axis=1)

In [8]:
# Initial testing of 400 rows
#df = df.loc[:400,:]

In [ ]:
# set(df.description_x_clean[0])

In [ ]:
# set(df.description_x_clean[0]).intersection(set(df.description_y_clean[0]))

In [10]:
# Feature engineering: intersecting bag of words
def intersection_BOW(df):
    x = df['description_x_clean']
    y = df['description_y_clean']

    return ' '.join(set(x).intersection(set(y))).split(' ')

In [11]:
# Feature engineering: Bag of words in either x or y but not both
def sym_diff_BOW(df):
    x = df['description_x_clean']
    y = df['description_y_clean']
    
    return ' '.join(set(x).symmetric_difference(set(y))).split(' ')

In [12]:
df['intersect_BOW'] = df[['description_x_clean','description_y_clean']].apply(intersection_BOW,axis=1)

In [13]:
df['sym_diff_BOW'] = df[['description_x_clean','description_y_clean']].apply(sym_diff_BOW,axis=1)

In [14]:
df.head()

,itemID,itemID_2,description_x_clean,description_y_clean,intersect_BOW,sym_diff_BOW
0,1,4112648,"[продам, камаз, 6520, 20, тонн]","[продам, камаз, 6520, 20, тонн]","[6520, камаз, продам, тонн, 20]",[]
1,3,1991275,[тюнинге],"[тюнинге, возможен, торг]",[тюнинге],"[возможен, торг]"
2,4,1223296,"[телефон, хорошем, состоянии, трещин, сколов, ...","[отличном, состоянии, комплекте, зарядник, каб...","[состоянии, комплекте, кабель]","[работает, держит, лежит, зарядник, родное, ин..."
3,7,1058851,"[отличный, подарок, новый, китайской, apple, н...","[отличный, подарок, новый, китайской, apple, н...","[245ггц, икпорт, мгц, использование, аккумулят...",[]
4,8,2161930,"[лыжные, ботинки, хорошем, состоянии, 34, размер]","[ботинки, 34, размер, хорошем, состоянии]","[ботинки, состоянии, размер, хорошем, 34]",[лыжные]


In [15]:
df.to_csv('intersectingBOW.csv', encoding='utf-8', index=False)

In [54]:
print('Score for CountVectorizer and SGD:')
print(cross_val_score(sgd_count, x, y, cv=6, scoring='accuracy').mean())

Score for CountVectorizer and SGD:
0.776050673548


In [40]:
y = df.isDuplicate
x = df.intersect_BOW

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
count_vect = CountVectorizer()
tfidf_vect = TfidfVectorizer()

In [36]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
nb = MultinomialNB()

In [37]:
nb_count = make_pipeline(count_vect, nb)
nb_tfidf = make_pipeline(tfidf_vect, nb)

In [ ]:
import pickle

 TODO Run GridSearch to find best parameters for best model
 Compute for both intersection and difference 


In [ ]:
nb_count.fit(x.loc[:50000,],y.loc[:50000,])
fn = 'nb_count.sav'
pickle.dump(nb_tfidf, open(fn), 'wb')

In [ ]:
nb_tfidf.fit(x.loc[:50000,],y.loc[:50000,])
fn = 'nb_tfidf.sav'
pickle.dump(nb_tfidf, open(fn), 'wb')
# Using this for feature
#nb_tfidf.predict_proba(x.loc[350:,])

In [49]:
# Scoring metrics
scores= ['accuracy', 'precision', 'recall', 'roc_auc']

In [61]:
print('Score for CountVectorizer and NB:')
# Accuracy: ratio of correctly predicted observation to total observations
# TP+TN /TP+FP+TN+FN
print(cross_val_score(nb_count, x, y, cv=6, scoring='accuracy').mean())

# ROC_AUC: High TP, Low FP
print(cross_val_score(nb_count, x, y, cv=6, scoring='roc_auc').mean())

# Precision: TP/(TP+FP)
# Of all passengers that labeled as survived, how many actually survived?
print(cross_val_score(nb_count, x, y, cv=6, scoring='precision').mean())

# Recall: TP/TP+FN
#  Of all the passengers that truly survived, how many did we label?
print(cross_val_score(nb_count, x, y, cv=6, scoring='recall').mean())

# F1 score: weighted average of Precision and Recall
# 2*(Recall * Precision) / (Recall + Precision)

Score for CountVectorizer and NB:
0.827955809189
0.8610410831
0.880952852179
0.878707482993


F1 is usually more useful than accuracy, especially if you have an uneven class distribution. Accuracy works best if false positives and false negatives have similar cost. If the cost of false positives and false negatives are very different, it’s better to look at both Precision and Recall. 

In [70]:
print('Score for tfidf and NB:')
print(cross_val_score(nb_tfidf, x, y, cv=6, scoring='accuracy').mean())
print(cross_val_score(nb_tfidf, x, y, cv=6, scoring='roc_auc').mean())

Score for tfidf and NB:
0.805680820496
0.852186430128


In [43]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [44]:
knn_count = make_pipeline(count_vect, knn)
knn_tfidf = make_pipeline(tfidf_vect, knn)

In [71]:
print('Score for CountVectorizer and KNN:')
print(cross_val_score(knn_count, x, y, cv=6, scoring='accuracy').mean())
print(cross_val_score(knn_count, x, y, cv=6, scoring='roc_auc').mean())

Score for CountVectorizer and KNN:
0.774150637188
0.849362522787


In [46]:
print('Score for tfidf and KNN:')
print(cross_val_score(knn_tfidf, x, y, cv=6, scoring='accuracy').mean())

Score for l1 and KNN:
0.771550003991


In [52]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()

In [53]:
sgd_count = make_pipeline(count_vect, sgd)
sgd_tfidf = make_pipeline(tfidf_vect, sgd)

In [73]:
print('Score for CountVectorizer and SGD:')
print(cross_val_score(sgd_count, x, y, cv=6, scoring='accuracy').mean())
print(cross_val_score(sgd_count, x, y, cv=6, scoring='roc_auc').mean())
print(cross_val_score(sgd_count, x, y, cv=6, scoring='precision').mean())
print(cross_val_score(sgd_count, x, y, cv=6, scoring='recall').mean())

Score for CountVectorizer and SGD:
0.783516685734
0.885251767374
0.856796134743
0.879319727891


In [55]:
print('Score for TfidfVectorizer and SGD:')
print(cross_val_score(sgd_tfidf, x, y, cv=6, scoring='accuracy').mean())

Score for TfidfVectorizer and SGD:
0.771035641756


In [77]:
from sklearn.model_selection import GridSearchCV

In [75]:
# Parameters for GridSearch
param_grid_nb ={'multinomialnb__alpha':[0,0.025,0.05,0.1,0.3]}
param_grid_nb_tfidf = {'multinomialnb__alpha':[0,0.025,0.05,0.1,0.3]
                       ,'tfidfvectorizer__norm':['l1','l2']
                       ,'tfidfvectorizer__min_df':[0.0,0.03,0.1]
                       ,'tfidfvectorizer__use_idf':[True,False]}

In [78]:
# Tfidf Naive Bayes
grid_nb_tfidf = GridSearchCV(nb_tfidf, param_grid_nb_tfidf, cv=5, scoring='accuracy')
grid_nb_tfidf.fit(x,y)

print(grid_nb_tfidf.best_score_)
print(grid_nb_tfidf.best_params_)

/home/juan/anaconda2/lib/python2.7/site-packages/sklearn/naive_bayes.py:699: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc) -
/home/juan/anaconda2/lib/python2.7/site-packages/sklearn/naive_bayes.py:700: RuntimeWarning: divide by zero encountered in log
  np.log(smoothed_cc.reshape(-1, 1)))
/home/juan/anaconda2/lib/python2.7/site-packages/sklearn/naive_bayes.py:700: RuntimeWarning: invalid value encountered in subtract
  np.log(smoothed_cc.reshape(-1, 1)))


0.783042394015
{u'tfidfvectorizer__norm': u'l1', u'multinomialnb__alpha': 0.1, u'tfidfvectorizer__min_df': 0.0, u'tfidfvectorizer__use_idf': False}


In [80]:
grid_nb_count = GridSearchCV(nb_count, param_grid_nb, cv=5, scoring='accuracy')
%time grid_nb_count.fit(x,y)

print(grid_nb_count.best_score_)
print(grid_nb_count.best_params_)

CPU times: user 432 ms, sys: 4 ms, total: 436 ms
Wall time: 512 ms
0.765586034913
{u'multinomialnb__alpha': 0.025}


In [81]:
from sklearn.model_selection import RandomizedSearchCV

In [82]:
# Tfidf Naive Bayes
random_nb_tfidf = RandomizedSearchCV(nb_tfidf, param_grid_nb_tfidf, cv=5, scoring='accuracy', n_iter=5, random_state=1)
random_nb_tfidf.fit(x,y)
print(random_nb_tfidf.best_score_)
print(random_nb_tfidf.best_params_)

0.753117206983
{u'tfidfvectorizer__norm': u'l2', u'multinomialnb__alpha': 0.1, u'tfidfvectorizer__min_df': 0.0, u'tfidfvectorizer__use_idf': False}
